In [4]:
import boto3
from dotenv import load_dotenv
import os
import re
import paramiko
import requests

In [5]:
load_dotenv()
aws_access_key_id= os.getenv("aws_access_key_id")
aws_secret_access_key = os.getenv("aws_secret_access_key")
region_name = os.getenv("region_name")

# Let's use Amazon S3
ec2 = boto3.client('ec2', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key, region_name=region_name)

In [7]:
EC2_KWARGS_MASTER = {
    "ImageId": "ami-07151644aeb34558a",
    "InstanceType": "t3.small", # t3.medium
    "MinCount": 1,
    "MaxCount": 1,
    "KeyName": "Ec2-boto3",  
    "SecurityGroupIds": ["sg-0b7b73b05e7577a3b"],
    "IamInstanceProfile":{
        'Arn': "arn:aws:iam::103976228435:instance-profile/EC2_S3_allowens"
    },
    "SubnetId":"subnet-0e19e3f6d404a4467",
    "UserData": f"""#!/bin/bash
                sudo yum update -y
                sudo yum install docker -y
                sudo service docker start
                sudo usermod -a -G docker ec2-user
                sudo docker pull taraman12/api-preprocessing:latest
                sudo docker pull taraman12/solar-park-detection-preprocessing:latest
                sudo docker pull taraman12/solar-park-detection-ml-serve:latest
             """, 
    "TagSpecifications": [
        {
            "ResourceType": "instance",
            "Tags": [
                {"Key": "Name", "Value": "master-instance"},
            ],
        },
    ],
}

In [21]:
EC2_KWARGS_WORKER = {
    "ImageId": "ami-07151644aeb34558a",
    "InstanceType": "t3.medium", # t3.medium
    "MinCount": 1,
    "MaxCount": 1,
    "KeyName": "Ec2-boto3",  
    "SecurityGroupIds": ["sg-0b7b73b05e7577a3b"],
    "IamInstanceProfile":{
        'Arn': "arn:aws:iam::103976228435:instance-profile/EC2_S3_allowens"
    },
    "SubnetId":"subnet-0e19e3f6d404a4467",
    "UserData": f"""#!/bin/bash
                sudo yum update -y
                sudo yum install docker -y
                sudo service docker start
                sudo usermod -a -G docker ec2-user
             """, 
    "TagSpecifications": [
        {
            "ResourceType": "instance",
            "Tags": [
                {"Key": "Name", "Value": "worker-instance"},
            ],
        },
    ],
}

In [17]:
EC2_KWARGS= {
    "ImageId": "ami-07151644aeb34558a",
    "InstanceType": "t3.small",
    "MinCount": 1,
    "MaxCount": 1,
    "KeyName": "Ec2-boto3",
    "SecurityGroupIds": ["sg-0b7b73b05e7577a3b"],
    "IamInstanceProfile":{
        'Arn': "arn:aws:iam::103976228435:instance-profile/EC2_S3_allowens"
    },
    "SubnetId":"subnet-0e19e3f6d404a4467",
    "UserData": f"""#!/bin/bash
                sudo yum update -y
                sudo yum install docker -y
                sudo service docker start
                sudo usermod -a -G docker ec2-user
                sudo docker pull taraman12/api-ml-serve:latest
                sudo docker pull taraman12/api-preprocessing:latest
                sudo docker swarm join --token {JOIN_TOKEN} {MANAGER_IP}:2377
                sudo docker service create -d \
                    -p 8080:8080 \
                    --network mynetwork \
                    --name ml-serve \
                    taraman12/api-ml-serve:latest
                sudo docker service create -d \
                    -v /var/run/docker.sock:/var/run/docker.sock \
                    -p 8000:8000 \
                    --network mynetwork \
                    --name preprocessing-w \
                    -e PASSWORD={password} \
                    -e JOIN_TOKEN={JOIN_TOKEN} \
                    -e MANAGER_IP={MANAGER_IP} \
                    taraman12/api-preprocessing:latest
             """,
    "TagSpecifications": [
        {
            "ResourceType": "instance",
            "Tags": [
                {"Key": "Name", "Value": "main-instance"},
            ],
        },
    ],
}

In [8]:
response = ec2.run_instances(**EC2_KWARGS_MASTER)

# ID der gestarteten Instanz abrufen
instance_id = response['Instances'][0]['InstanceId']
print(f'Started instance {instance_id}')

Started instance i-044ffcc6e63d230da


In [22]:
response = ec2.run_instances(**EC2_KWARGS_WORKER)

# ID der gestarteten Instanz abrufen
instance_id = response['Instances'][0]['InstanceId']
print(f'Started instance {instance_id}')

Started instance i-01710afbae23e3123


In [11]:
response['Instances']

[{'AmiLaunchIndex': 0,
  'ImageId': 'ami-07151644aeb34558a',
  'InstanceId': 'i-076f715dbd92167af',
  'InstanceType': 't3.small',
  'KeyName': 'Ec2-boto3',
  'LaunchTime': datetime.datetime(2023, 11, 30, 19, 41, 44, tzinfo=tzutc()),
  'Monitoring': {'State': 'disabled'},
  'Placement': {'AvailabilityZone': 'eu-central-1a',
   'GroupName': '',
   'Tenancy': 'default'},
  'PrivateDnsName': 'ip-172-31-25-64.eu-central-1.compute.internal',
  'PrivateIpAddress': '172.31.25.64',
  'ProductCodes': [],
  'PublicDnsName': '',
  'State': {'Code': 0, 'Name': 'pending'},
  'StateTransitionReason': '',
  'SubnetId': 'subnet-0e19e3f6d404a4467',
  'VpcId': 'vpc-07332de596f81bf40',
  'Architecture': 'x86_64',
  'BlockDeviceMappings': [],
  'ClientToken': 'c26df8e9-260d-4a7c-abc0-36fde4f3b762',
  'EbsOptimized': False,
  'EnaSupport': True,
  'Hypervisor': 'xen',
  'IamInstanceProfile': {'Arn': 'arn:aws:iam::103976228435:instance-profile/EC2_S3_allowens',
   'Id': 'AIPARQNLXKZJSVFPURJ24'},
  'NetworkIn